In [1]:
# DAY 01
nums = [int(x) for x in open("../data/01.txt").readlines()]

def sliding_window(arr, n):
    for i in range(0, len(arr) - n + 1):
        yield arr[i:i+n]

res1 = sum(x[1] > x[0] for x in sliding_window(nums, 2))
res2 = sum(x[1] > x[0] for x in sliding_window(
    [sum(y) for y in sliding_window(nums, 3)], 
    2))

print(f'Answer 1: {res1}\nAnswer 2: {res2}')

Answer 1: 1557
Answer 2: 1608


In [2]:
# DAY 02
from functools import reduce

commands = [(x.split()[0], int(x.split()[1])) 
             for x in open("../data/02.txt").readlines()]

def step1(pos, command):
    (x, depth) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth - offs)
    elif cmd == 'down':
        return (x, depth + offs)
    elif cmd == 'forward':
        return (x + offs, depth)
    
def step2(pos, command):
    (x, depth, aim) = pos
    (cmd, offs) = command
    if cmd == 'up':
        return (x, depth, aim - offs)
    elif cmd == 'down':
        return (x, depth, aim + offs)
    elif cmd == 'forward':
        return (x + offs, depth + aim * offs, aim)
        
pos1 = reduce(step1, commands, (0, 0))
pos2 = reduce(step2, commands, (0, 0, 0))
print(f'Answer 1: {pos1[0] * pos1[1]}\nAnswer 2: {pos2[0] * pos2[1]}')

Answer 1: 1728414
Answer 2: 1765720035


In [3]:
# DAY 03
def get_most_least_common(nums, pos):
    n0 = 0
    n1 = 0
    for num in nums:
        n0 += num[pos] == '0'
        n1 += num[pos] == '1'
    return ('1', '0') if n1 >= n0 else ('0', '1')

def part1(nums):
    lc, mc = "", ""
    for i in range(0, len(nums[0])):
        (m, l) = get_most_least_common(nums, i)
        mc += m
        lc += l
    return int(mc, 2) * int(lc, 2)
     
def part2(nums):
    mnums = nums[:]
    lnums = nums[:]
    for i in range(0, len(nums[0])):
        if len(mnums) > 1:
            (m, l) = get_most_least_common(mnums, i)
            mnums = [x for x in mnums if x[i] == m]

        if len(lnums) > 1:
            (m, l) = get_most_least_common(lnums, i)
            lnums = [x for x in lnums if x[i] == l]

    return int(mnums[0], 2) * int(lnums[0], 2)

nums = [x.strip() for x in open('../data/03.txt').readlines()]
print(f'Answer 1: {part1(nums)}\nAnswer 2: {part2(nums)}')

Answer 1: 2972336
Answer 2: 3368358


In [96]:
# DAY 04
def get_boards(data):
    cells = []
    for line in data:
        if line == '':
            yield cells
            cells = []
        else:
            cells.append([int(x) for x in line.split()])
    yield cells

def is_full(board, pos, is_vert):
    for i in range(len(board)):
        n = board[i][pos] if is_vert else board[pos][i]
        if n != -1:
            return False
    return True

def has_winner(board):
    return any(map(lambda i: 
                       is_full(board, i, True) or is_full(board, i, False), 
                   range(len(board))))

def apply_num(board, num):
    for row in board:
        for i, el in enumerate(row):
            if el == num:
                row[i] = -1
            
def run_simulation(nums, boards):
    winners, scores = [], []
    for n in nums:
        for i, board in enumerate(boards):
            if i in winners:
                continue
            apply_num(board, n)
            if has_winner(board):
                s = n * sum(x for row in board for x in row if x != -1)
                winners.append(i)
                scores.append(s)
    return scores
             
lines = [x.strip() for x in open('../data/04.txt').readlines()]
nums = [int(x) for x in lines[0].split(",")]
boards = list(get_boards(lines[2:]))
scores = run_simulation(nums, boards)

print(f"Answer 1: {scores[0]}\nAnswer 2: {scores[-1]}")

Answer 1: 44736
Answer 2: 1827


In [9]:
# DAY 05
from collections import defaultdict

def sgn(x):
    if x == 0:
        return 0
    return 1 if x > 0 else -1

def stroke(a, b, ptmap, skip_diagonals):
    (x1, y1) = a
    (x2, y2) = b
    dx = sgn(x2 - x1)
    dy = sgn(y2 - y1)
    
    if skip_diagonals and dx != 0 and dy != 0:
        return
    
    cx, cy = x1, y1
    while True:
        ptmap[(cx, cy)] += 1
        if cx == x2 and cy == y2:
            break
        cx += dx
        cy += dy
        
def find_num_overlaps(points, skip_diagonals):
    ptmap = defaultdict(int)
    for a, b in points:
        stroke(a, b, ptmap, skip_diagonals)
    return sum(ptmap[x] >= 2 for x in ptmap)

def parse_pt(pt):
    (x, y) = [int(c) for c in pt.split(',')]
    return (x, y)

points = [list(map(parse_pt, x.strip().split(' -> ')))
          for x in open('../data/05.txt').readlines()]
res1 = find_num_overlaps(points, True)
res2 = find_num_overlaps(points, False)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 7436
Answer 2: 21104


In [387]:
# DAY 06
def sum_spawned(nums, total_days):
    spawned = {}

    def get_n_spawned(n, days):
        if days <= n:
            return 0
        if (n, days) in spawned:
            return spawned[(n, days)]
        res = (days - n - 1) // 7 + 1

        for i in range(res):
            res += get_n_spawned(8, days - n - i * 7 - 1)
        spawned[(n, days)] = res
        return res

    return sum(get_n_spawned(x, total_days) for x in nums) + len(nums)

nums = [int(x) for x in open('../data/06.txt').read().split(',')]
print(f"Answer 1: {sum_spawned(nums, 80)}\nAnswer 2: {sum_spawned(nums, 256)}")

Answer 1: 386755
Answer 2: 1732731810807


In [19]:
# DAY 07
import statistics

def part1(nums):
    c = statistics.median(nums)
    return sum(abs(x - c) for x in nums)

# simple gradient descent
def fmin(p0, f, num_it=1000):
    EPS = 0.001
    p = p0
    for i in range(num_it):
        pr = f(p + EPS)
        pl = f(p - EPS)
        dp = (pr - pl)/(2 * EPS)
        p = p - dp * EPS
    return p

def part2(nums):
    def f(x):
        res = 0
        for n in nums:
            d = abs(x - n)
            res += d * (d + 1)/2
        return res
    minx = int(round(fmin(statistics.median(nums), f)))
    return f(minx)

nums = [int(x) for x in open('../data/07.txt').read().split(',')]
print(f"Answer 1: {int(part1(nums))}\nAnswer 2: {int(part2(nums))}")

Answer 1: 343468
Answer 2: 96086265


In [ ]:
# DAY08
CHARS = 'abcdefg'
DIGITS = ['abcefg', 'cf', 'acdeg', 'acdfg', 'bcdf',
         'abdfg', 'abdefg', 'acf', 'abcdefg', 'abcdfg']
DIGITS_MAP = {x: i for i, x in enumerate(DIGITS)}
NUM_SEGMENTS = 7

def normalize_mapping(mapping):
    changed = True
    while changed:
        changed = False
        for i, c in enumerate(mapping):
            if sum(c) == 1:
                p = c.index(1)
                for j in range(NUM_SEGMENTS):
                    if j != i and mapping[j][p] == 1:
                        mapping[j][p] = 0
                        changed = True
                        
def is_valid_mapping(mapping):
    if not all(sum(c) == 1 for c in mapping):
        return False
    for i in range(NUM_SEGMENTS):
        s = [mapping[j][i] for j in range(NUM_SEGMENTS)]
        if sum(s) != 1:
            return False
    return True

def get_mapping(mapping, combs, pos):
    print(f"mapping: {mapping} pos: {pos}")
    if pos == len(combs):
        normalize_mapping(mapping)
        return mapping if is_valid_mapping(mapping) else None
    comb = combs[pos]
    for digit in DIGITS:
        if len(digit) != len(comb):
            continue
        print(digit, comb)
        mapping1 = [x[:] for x in mapping]
        for d in [x for x in CHARS if x not in digit]: 
            for c in comb:
                mapping1[ord(c) - ord('a')][ord(d) - ord('a')] = 0
        res_mapping = get_mapping(mapping1, combs, pos + 1)
        if res_mapping != None:
            return res_mapping
    return None
  
def decode_digit(m, combs):
    res = 0
    for comb in combs:
        s = "".join(sorted(chr(m[ord(c) - ord('a')].index(1) + ord('a')) for c in comb))
        if s not in DIGITS_MAP:
            return None
        digit = DIGITS_MAP[s]
        res = res * 10 + digit
    return res

def decode_sum(inp):
    res = 0
    for combs, to_decode in inp:
        mapping = [[1] * NUM_SEGMENTS] * NUM_SEGMENTS
        combs.sort(key=len)
        mapping = get_mapping(mapping, combs, 0)
        res += decode_digit(mapping, to_decode)
    return res

def parse(line):
    return line.strip().split(" ")

inp = [list(map(parse, x.strip().split("|"))) 
       for x in open('../data/08.txt').readlines()]

res1 = sum(len(x) in [2, 3, 4, 7] for a, b in inp for x in b)
res2 = decode_sum(inp)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

In [315]:
# DAY09
import math

OFFS = [(0, 1), (0, -1), (1, 0), (-1, 0)]

def find_minima(board):
    res = []
    for j, row in enumerate(board):
        for i, h in enumerate(row):
            is_min = True
            for dx, dy in OFFS:
                x, y = i + dx, j + dy
                if x >= 0 and x < len(row) and y >= 0 and y < len(board) and board[y][x] <= h:
                    is_min = False
                    break
            if is_min:
                res.append((i, j))
    return res
    
def fill_cell(board, bidx, pos, basin_idx):
    x, y = pos
    if x < 0 or x >= len(board[0]) or y < 0 or y >= len(board):
        return
    b = board[y][x]
    if b == 9:
        bidx[y][x] = 0
        return
        
    if bidx[y][x] >= 0:
        return
    bidx[y][x] = basin_idx
    h = board[y][x]
    for dx, dy in OFFS:
        x1, y1 = x + dx, y + dy
        fill_cell(board, bidx, (x1, y1), basin_idx)

def find_basins(board):
    w, h = len(board[0]), len(board)
    bidx = [[-1]*w for i in range(h)]
    num_basins = 0
    for j in range(h):
        for i in range(w):
            if bidx[j][i] == -1:
                fill_cell(board, bidx, (i, j), num_basins + 1)
                num_basins += 1
    return bidx
                
board = [list(map(int, list(x.strip()))) for x in open('../data/09.txt').readlines()]

mins = find_minima(board)
res1 = sum(map(lambda p: board[p[1]][p[0]] + 1, mins))

basins = find_basins(board)
max_basin = max(map(max, basins))

basin_sizes = []
for bidx in range(1, max_basin + 1):
    size = 0
    w, h = len(board[0]), len(board)
    for j in range(h):
        for i in range(w):
            if basins[j][i] == bidx:
                size += 1
    basin_sizes.append(size)

res2 = math.prod(sorted(basin_sizes, reverse=True)[:3])
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 588
Answer 2: 964712


In [159]:
# DAY10
CMAP = {"[": "]", "{": "}", "<": ">", "(": ")"}
SCORES1 = {")": 3, "]": 57, "}": 1197, ">": 25137}
SCORES2 = {")": 1, "]": 2, "}": 3, ">": 4}
CORRUPTED = 1
INCOMPLETE = 2

def find_match(line):
    chars = []
    for c in line:
        if c in "[({<":
            chars.append(c)
        else:
            current = chars.pop()
            if c != CMAP[current]:
                return (CORRUPTED, c)
    return (INCOMPLETE, "".join(CMAP[x] for x in chars[::-1]))

def score2(line):
    res = 0
    for c in line:
        res = res * 5 + SCORES2[c]
    return res

lines = [x.strip() for x in open('../data/10.txt').readlines()]
matches = [find_match(line) for line in lines]
res1 = sum(SCORES1[x] for (t, x) in matches if t == CORRUPTED)
scores2 = sorted(score2(x) for (t, x) in matches if t == INCOMPLETE)
res2 = scores2[len(scores2) // 2]
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 216297
Answer 2: 2165057169


In [265]:
# DAY11

OFFS = [(1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0), (-1, -1), (0, -1), (1, -1)] 

def flash(board, x, y):
    res = 1
    board[y][x] == 11
    for dx, dy in OFFS:
        cx, cy = x + dx, y + dy
        if cx >= 0 and cx < len(board[0]) and cy >= 0 and cy < len(board):
            board[cy][cx] += 1            
            if board[cy][cx] == 10:
                res += flash(board, cx, cy)
    return res

def step(board):
    num_flashes = 0
    for j in range(len(board)):
        for i in range(len(board[0])):
            board[j][i] += 1
            if board[j][i] == 10:
                num_flashes += flash(board, i, j)    
    for j in range(len(board)):
        for i in range(len(board[0])): 
            if board[j][i] >= 10:
                board[j][i] = 0
    return num_flashes
    
board = [list(map(int, list(s))) for s in open("../data/11.txt").read().split()]

res1 = 0
board1 = [row[:] for row in board]
for i in range(100):
    res1 += step(board1)

res2 = 0
while True:
    num_flashes = step(board)
    res2 += 1
    if num_flashes == len(board) * len(board[0]):
        break

print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 1683
Answer 2: 788


In [314]:
# DAY12
from collections import defaultdict

def traverse1(graph, pos, visited):    
    res = 0
    for next_pos in graph[pos]:
        if next_pos == "end":
            res += 1
            continue
        if next_pos == "start":
            continue
        if next_pos.islower() and visited[next_pos] == 1:
            continue
        visited[next_pos] += 1
        res += traverse1(graph, next_pos, visited)
        visited[next_pos] -= 1
    return res
        
def traverse2(graph, pos, visited, visited_small=None):    
    res = 0
    for next_pos in graph[pos]:
        if next_pos == "end":
            res += 1
            continue
        if next_pos == "start":
            continue
        if next_pos.islower():
            if visited[next_pos] == 1:
                if visited_small == None or visited_small == next_pos:
                    visited[next_pos] += 1
                    res += traverse2(graph, next_pos, visited, next_pos)
                    visited[next_pos] -= 1
                continue
            if visited[next_pos] == 2:
                continue
        visited[next_pos] += 1
        res += traverse2(graph, next_pos, visited, visited_small)
        visited[next_pos] -= 1
    return res

edges = [line.strip().split("-") for line in open("../data/12.txt").readlines()]

verts = set()
for a, b in edges:
    verts.add(a)
    verts.add(b)
verts = list(verts)
    
graph = defaultdict(list)
for a, b in edges:
    graph[a].append(b)
    graph[b].append(a)
    
visited = {v: 0 for v in graph}
res1 = traverse1(graph, "start", visited)
res2 = traverse2(graph, "start", visited)

print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 4970
Answer 2: 137948


In [110]:
# DAY13
def fold(coords, dir, pos):
    res = []
    if dir == 'x':
        for x, y in coords:
            if x < pos:
                res.append((x, y))
            else:
                res.append((2*pos - x, y))
    else:
        for x, y in coords:
            if y < pos:
                res.append((x, y))
            else:
                res.append((x, 2*pos - y))
    return res
    
clines, flines = open('../data/13.txt').read().split("\n\n")
coords = [list(map(int, x.split(","))) for x in clines.split("\n")]
fdirs = [x[11:].split("=") for x in flines.strip().split("\n")]

coords = set(fold(coords, fdirs[0][0], int(fdirs[0][1])))
res1 = len(set(coords))

for dir, pos in fdirs[1:]:
    coords = set(fold(coords, dir, int(pos)))

mx, my = max(coords)
m = [['.']*(mx + 1) for i in range(my + 1)] 
for x, y in coords:
    m[y][x] = "#"
res2 = "\n".join("".join(x) for x in m)
print(f"Answer 1: {res1}\nAnswer 2:\n{res2}")

Answer 1: 708
Answer 2:
####.###..#....#..#.###..###..####.#..#
#....#..#.#....#..#.#..#.#..#.#....#..#
###..###..#....#..#.###..#..#.###..####
#....#..#.#....#..#.#..#.###..#....#..#
#....#..#.#....#..#.#..#.#.#..#....#..#
####.###..####..##..###..#..#.#....#..#


In [232]:
# DAY14
from collections import defaultdict

def step(hist, rules):
    res = defaultdict(int)
    for s, cnt in hist.items():
        if s in rules:
            c = rules[s]
            res[s[0] + c] += cnt
            res[c + s[1]] += cnt
        else:
            res[s] = cnt
    return res

def get_pairs_hist(pattern):
    res = defaultdict(int)
    for i in range(1, len(pattern)):
        res[pattern[i - 1: i + 1]] += 1
    return res

def eval_pattern(start_pattern, rules, steps):
    hist = get_pairs_hist(start_pattern)
    for i in range(steps):
        hist = step(hist, rules)
        
    char_hist = defaultdict(int)
    for s, cnt in hist.items():
        char_hist[s[0]] += cnt
    char_hist[start_pattern[-1]] += 1

    counts = sorted((char_hist[c], c) for c in char_hist.keys())
    return counts[-1][0] - counts[0][0]
    
lines = open('../data/14.txt').readlines()
start_pattern = lines[0].strip()
rules = {x[0]: x[1] for x in map(lambda s: s.strip().split(" -> "), lines[2:])}

res1 = eval_pattern(start_pattern, rules, 10)
res2 = eval_pattern(start_pattern, rules, 40)
print(f"Answer 1: {res1}\nAnswer 2: {res2}")

Answer 1: 2621
Answer 2: 2843834241366
